In [61]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt


In [62]:
def multinacional_metric(y_true, y_pred):
     return abs(sum(y_true - y_pred)) / sum(y_true)

In [63]:
df_tb_productos = pd.read_csv('data/tb_productos.txt', sep='\t')
df_sell_in = pd.read_csv('data/sell-in.txt', sep='\t')
df_predecir = pd.read_csv('data/productos_a_predecir.txt', sep='\t')
df_tb_stocks = pd.read_csv('data/tb_stocks.txt', sep='\t')

In [64]:
# Join tb_productos to sell_in on product_id
df_sell_in_merged = pd.merge(df_sell_in, df_tb_productos, on='product_id', how='left')
# Join tb_stocks to sell_in_merged on both product_id and periodo
df_final = pd.merge(df_sell_in_merged, df_tb_stocks, on=['product_id', 'periodo'], how='left')
df_final.head(10)

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,201701,10125,20524,0,1,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,201701,10012,20524,0,11,1.54452,1.54452,HC,VAJILLA,Cristalino,Importado,500.0,NaN
5,201701,10080,20524,0,1,0.01514,0.01514,HC,VAJILLA,Cristalino,Importado,500.0,NaN
6,201701,10015,20524,0,4,0.10600,0.10600,HC,VAJILLA,Cristalino,Importado,500.0,NaN
7,201701,10062,20524,0,1,0.18928,0.18928,HC,VAJILLA,Cristalino,Importado,500.0,NaN
8,201701,10159,20524,0,3,0.02271,0.02271,HC,VAJILLA,Cristalino,Importado,500.0,NaN
9,201701,10183,20524,0,1,0.01514,0.01514,HC,VAJILLA,Cristalino,Importado,500.0,NaN


In [65]:
df_final.columns

Index(['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados',
       'cust_request_qty', 'cust_request_tn', 'tn', 'cat1', 'cat2', 'cat3',
       'brand', 'sku_size', 'stock_final'],
      dtype='object')

In [66]:
df_final['fecha'] = pd.to_datetime(df_final['periodo'], format='%Y%m')
all_periods = pd.DataFrame()
all_periods['fecha'] = pd.date_range(start='2017-01-01', periods=36, freq='MS')
df_final = pd.merge(all_periods, df_final, on=['fecha'], how='left')

In [67]:
df_final.head(3)

,fecha,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,2017-01-01,201701,10234,20524,0,2,0.05300,0.05300,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,2017-01-01,201701,10032,20524,0,1,0.13628,0.13628,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,2017-01-01,201701,10217,20524,0,1,0.03028,0.03028,HC,VAJILLA,Cristalino,Importado,500.0,NaN


In [68]:
df_final = df_final[df_final['product_id'].isin(df_predecir['product_id'])]
df_final = df_final[df_final['fecha'] > '2018-01-01']
df_final.set_index('fecha', inplace=True)
df_final.index = df_final.index.to_period('M')
df_final.fillna(0, inplace=True)
df_final = df_final.groupby(['fecha','product_id']).agg({'tn': 'sum'}).reset_index()



In [69]:
df_final.set_index('fecha', inplace=True)
df_final.head(3)

,product_id,tn
fecha,,
2018-02,20001,1043.76470
2018-02,20002,712.00087
2018-02,20003,788.30749


In [74]:
predictions = []
# Ajustar y predecir para cada producto
for producto in df_final['product_id'].unique():
    # Filtrar los datos del producto
    df_producto = df_final[df_final['product_id'] == producto]
    df_producto = df_producto['tn']

    # Ajustar el modelo ARIMA
    model = ARIMA(df_producto, order=(2, 2, 6)) 
    model_fit = model.fit()

    # Realizar el pronóstico para 2 meses adelante
    forecast_steps = 2
    forecast = model_fit.forecast(steps=forecast_steps)
    
    # Obtener la predicción del segundo mes
    second_month_prediction = forecast.iloc[1]  # .iloc[1] obtiene el segundo valor predicho

    # Almacenar el producto_id y la predicción en la lista
    predictions.append({'product_id': producto, 'prediccion_2_mes': second_month_prediction})
    
   

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Ad

In [75]:
# Convertir la lista a un DataFrame
df_predictions = pd.DataFrame(predictions)

In [76]:

df_predictions.rename(columns={'product_id': 'product_id', 'prediccion_2_mes': 'tn'}, inplace=True)
df_predictions.to_csv('data/predicciones.csv', index=False,header=True)
df_predictions.head(10)


,product_id,tn
0,20001,1626.964987
1,20002,1625.612606
2,20003,910.374155
3,20004,765.206886
4,20005,821.853694
5,20006,355.370798
6,20007,363.124734
7,20008,288.764470
8,20009,563.322343
9,20010,982.914544


In [ ]:
#PRIMERA PARTE

In [ ]:
df_marzo = df_sell_in[df_sell_in['periodo']==201903]

In [ ]:
df_mayo = df_sell_in[df_sell_in['periodo']==201905]

In [ ]:
mayo_tn = df_mayo.groupby('product_id')['tn'].sum().reset_index()

In [ ]:
marzo_tn = df_marzo.groupby('product_id')['tn'].sum().reset_index()

In [ ]:
mayo_tn = mayo_tn.rename(columns={'tn': 'tn_actual'})
marzo_tn = marzo_tn.rename(columns={'tn': 'tn_pred'})

In [ ]:
df_pred = pd.merge(mayo_tn, marzo_tn, how="left", on=["product_id"])

In [ ]:
multinacional_metric(df_pred['tn_actual'].fillna(0),df_pred['tn_pred'].fillna(0))

0.06281789350589442

In [ ]:
#USAR LOS ULTIMOS 6 MESES Y PROMEDIAR PARA ESTIMAR

In [ ]:
df_seismeses = df_sell_in[(df_sell_in['periodo']>=201810) & (df_sell_in['periodo']<=201903)]
df_seismeses = df_seismeses.groupby(['periodo','product_id'])['tn'].sum().reset_index()
promedio_por_producto = df_seismeses.groupby('product_id')['tn'].mean().reset_index()
promedio_por_producto.rename(columns={'tn': 'tn_pred_seis'}, inplace=True)
df_pred_seis = pd.merge(mayo_tn, promedio_por_producto, how="left", on=["product_id"])
multinacional_metric(df_pred_seis['tn_actual'].fillna(0),df_pred_seis['tn_pred_seis'].fillna(0))

0.02615791041789335

In [ ]:
#SEGUNDA PARTE

In [ ]:
#PREDECIR FEBRERO 2020

In [ ]:
df_diciembre = df_sell_in[df_sell_in['periodo']==201912]

In [ ]:
df_diciembre = df_diciembre.groupby('product_id')['tn'].sum().reset_index()

In [ ]:
df_dic_feb = pd.merge(df_predecir, df_diciembre, how="left", on=["product_id"])

In [ ]:
df_dic_feb.to_csv('diciembre.csv',sep=',', index=False)  

In [ ]:
df_dic_feb

,product_id,tn
0,20001,1504.68856
1,20002,1087.30855
2,20003,892.50129
3,20004,637.90002
4,20005,593.24443
...,...,...
775,21263,0.01270
776,21265,0.05007
777,21266,0.05121
778,21267,0.01569


In [ ]:
#PREDECIR USANDO 6 MESES

In [ ]:
df_seismeses_2 = df_sell_in[(df_sell_in['periodo']>=201907) & (df_sell_in['periodo']<=201912)]
df_seismeses_2 = df_seismeses_2.groupby(['periodo','product_id'])['tn'].sum().reset_index()
seismeses_2 = df_seismeses_2.groupby('product_id')['tn'].mean().reset_index()
df_seismeses_2 = pd.merge(df_predecir, seismeses_2, how="left", on=["product_id"])
df_seismeses_2.to_csv('juniodiciembre.csv',sep=',', index=False)  

In [ ]:
#1 año MEJOR QUE DIÓ

In [ ]:
df_year = df_sell_in[(df_sell_in['periodo']>=201900) & (df_sell_in['periodo']<=201912)]
year_pred = df_year.groupby(['periodo','product_id'])['tn'].sum().reset_index()
year_pred = year_pred.groupby('product_id')['tn'].mean().reset_index()
year_pred = pd.merge(df_predecir, year_pred, how="left", on=["product_id"])
year_pred.to_csv('year2019.csv',sep=',', index=False)  

In [ ]:
#all
df_all = df_sell_in[df_sell_in['periodo']<=201912]
all_pred = df_all.groupby(['periodo','product_id'])['tn'].sum().reset_index()
all_pred = all_pred.groupby('product_id')['tn'].mean().reset_index()
all_pred = pd.merge(df_predecir, all_pred, how="left", on=["product_id"])
all_pred.to_csv('all2019.csv',sep=',', index=False)  

In [ ]:
#PRUEBAS!---

In [ ]:

df_febrero2019 = df_sell_in[df_sell_in['periodo']==201902] #SUPONGO QUE FEBRERO ES 2020
df_febrero2019 = df_febrero2019.groupby('product_id')['tn'].mean().reset_index()
df_febrero2019 = df_febrero2019.rename(columns={'tn': 'tn_actual'})

df_pred_2019 = pd.merge(df_febrero2019, all_pred, how="left", on=["product_id"])

multinacional_metric(df_pred_2019['tn_actual'].fillna(0),df_pred_2019['tn'].fillna(0))

125.24974374928281

In [ ]:
df_2017 = df_sell_in[df_sell_in['periodo']<=201712]
df_2017 = df_2017.groupby(['periodo','product_id'])['tn'].sum().reset_index()
df_2017 = df_2017.groupby('product_id')['tn'].mean().reset_index()
df_2017 = pd.merge(df_predecir, df_2017, how="left", on=["product_id"])
df_2017 = df_2017.fillna(0)
df_2017.to_csv('all2017.csv',sep=',', index=False)  

In [ ]:
df_2017

,product_id,tn
0,20001,1234.417246
1,20002,765.303166
2,20003,1007.067587
3,20004,733.870823
4,20005,686.620974
...,...,...
775,21263,0.000000
776,21265,0.000000
777,21266,0.000000
778,21267,0.000000
